In [1]:
import cv2 
import os 
import glob
import json
import csv
import io
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from datetime import datetime


from pathlib import Path
from functools import partial
from p_tqdm import p_map
from tqdm import tqdm 
from module.utils import imread, imfindcircles, findProjectiveTransform, str2array, circle_detection_multi_thread, find_valid_dest_circles
from module.disp_measure import displacement_measure, homography_transformation
from module.apca import run_apca_cas 

# Import circle detection and APCA parameters 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)



In [2]:
import json
with open('sensor_data.json') as f:
    sensors_dict = json.load(f)

In [4]:
data_fr = pd.DataFrame(sensors_dict)

In [5]:
data_fr

,093,094,095,097,098,099,103,104,105,107,...,201,203,204,205,207,208,209,211,212,213
type,Img,Img,Tmp,Img,Img,Tmp,Img,Img,Tmp,Img,...,Tmp,Img,Img,Tmp,Img,Img,Tmp,Img,Img,Tmp
pier,P1,P1,P1,P1,P1,P1,P9,P9,P9,P9,...,P55,P55,P55,P55,P59,P59,P59,P59,P59,P59
slab,1,1,1,1,1,1,8,8,8,9,...,54,55,55,55,58,58,58,58,58,58
block,1,1,1,1,1,1,1,1,1,2,...,9,10,10,10,10,10,10,10,10,10
snsr_dir,S,E,E,S,W,E,N,W,S,S,...,S,S,E,E,N,W,S,N,E,S
snsr_loc,S,S,S,S,S,S,E,E,E,S,...,E,S,S,S,E,E,E,E,E,E
disp_dir,FALSE,Forward,Forward,FALSE,Backward,Forward,FALSE,Forward,FALSE,FALSE,...,FALSE,FALSE,Forward,Forward,FALSE,Forward,FALSE,FALSE,Backward,FALSE
install_year,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020,...,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020
circles,"{'20200901': {'000100': [[824.707423580786, 59...","{'20200901': {'000100': [[789.7391304347826, 3...",{},"{'20200901': {'000100': [[878.4888888888889, 8...","{'20200901': {'000100': [[1012.0653846153846, ...",{},"{'20200901': {'090100': [[988.9198113207547, 4...","{'20200901': {'090100': [[1059.5321888412018, ...",{},"{'20200901': {'000100': [[1106.7730769230768, ...",...,{},"{'20200901': {'000100': [[865.0039215686274, 3...","{'20200901': {'000100': [[1060.1960784313726, ...",{},"{'20200901': {'000100': [[1005.0038759689922, ...","{'20200901': {'000100': [[845.6711711711712, 4...",{},"{'20200901': {'000100': [[1053.1115241635687, ...","{'20200901': {'000100': [[906.7540983606557, 4...",{}
x_disp,"{'20200901': {'000100': -0.4961069101654534, '...","{'20200901': {'000100': 2.8678571824450856, '0...",NaN,"{'20200901': {'000100': -0.014975959020063101,...","{'20200901': {'000100': -3.063497713549097, '0...",NaN,"{'20200901': {'090100': -0.3277675480277651, '...","{'20200901': {'090100': 1.6696462785624284, '1...",NaN,"{'20200901': {'000100': 0.11437187671751303, '...",...,NaN,"{'20200901': {'000100': 0.3291621478224869, '0...","{'20200901': {'000100': -0.518887632989717, '0...",NaN,"{'20200901': {'000100': -0.005990029379900008,...","{'20200901': {'000100': 2.905564052054686, '01...",NaN,"{'20200901': {'000100': -0.1313660287734102, '...","{'20200901': {'000100': -1.73328103022099, '01...",NaN


In [8]:
!pip install xlsxwriter

     |████████████████████████████████| 144 kB 568 kB/s eta 0:00:01


In [77]:
writer = pd.ExcelWriter('엑셀파일명.xlsx', engine='xlsxwriter')


In [78]:
sensor_info

,snsr_num,type,pier,slab,block,snsr_dir,snsr_loc,disp_dir,install_year
0,093,Img,P1,1,1,S,S,FALSE,2020
1,094,Img,P1,1,1,E,S,Forward,2020
2,095,Tmp,P1,1,1,E,S,Forward,2020
3,097,Img,P1,1,1,S,S,FALSE,2020
4,098,Img,P1,1,1,W,S,Backward,2020
...,...,...,...,...,...,...,...,...,...
87,208,Img,P59,58,10,W,E,Forward,2020
88,209,Tmp,P59,58,10,S,E,FALSE,2020
89,211,Img,P59,58,10,N,E,FALSE,2020
90,212,Img,P59,58,10,E,E,Backward,2020


In [79]:
sensor_info = pd.DataFrame(columns=['snsr_num','type','pier','slab','block',
                                          'snsr_dir', 'snsr_loc', 'disp_dir', 'install_year'])

for k, v in  data_fr.items() : 

    
    sensor_info = sensor_info.append({'snsr_num' : str(k) , 
                                      'type' : v['type'],
                                      'pier':v['pier'],
                                      'slab':v['slab'],
                                      'block':v['block'],
                                      'snsr_dir':v['snsr_dir'],
                                      'snsr_loc':v['snsr_loc'],
                                      'disp_dir':v['disp_dir'], 
                                      'install_year' : v['install_year']} , ignore_index = True)
    


sensor_info.to_excel(writer, sheet_name= 'snsr_info')


In [80]:
for k, v in  data_fr.items() : 
    
    
    
    if v['circles'] : 
        disp_info = pd.DataFrame(columns=['datetime', 'circles','x_disp','y_disp'])
    
        for k2, v2 in v['circles'].items():
            for k3, v3 in v2.items():

    
                disp_info = disp_info.append({'datetime' : str(k2) + str(k3[:2]) , 
                                                  'circles' : v3,
                                                  'x_disp':v['x_disp'][k2][k3],
                                                  'y_disp':v['y_disp'][k2][k3]} , ignore_index = True)
        disp_info.to_excel(writer, sheet_name= k)
        
    elif not v['temp'] != v['temp'] : 
        disp_info = pd.DataFrame(columns=['datetime', 'temp'])
    
        for k2, v2 in v['temp'].items():
            for k3, v3 in v2.items():

    
                disp_info = disp_info.append({'datetime' : str(k2) + str(k3[:2]) , 
                                                  'temp' : v3} , ignore_index = True)
        
        disp_info.to_excel(writer, sheet_name= k)
    
    
    
    

In [81]:
writer.save()